In [1]:
import zipfile
unzip = False
if unzip:
    with zipfile.ZipFile("Data.zip","r") as zip_ref:
        zip_ref.extractall("Data")

In [2]:
import os
train = 'Data/Data'
train_img = []
train_labels = []
for direct in os.listdir(train):
    pathi = train+'/'+direct
    print(direct)
    for img in os.listdir(pathi):
        train_img.append(pathi+'/'+img)
        if direct == 'NORMAL':
            train_labels.append(0)
        elif direct == 'COVID19':
            train_labels.append(1)
        else:
            train_labels.append(2)

PNEUMONIA
COVID19
NORMAL


In [3]:
train_img

['Data/Data/PNEUMONIA/PNEUMONIA(4093).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3903).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(4147).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(970).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(1408).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(1860).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3540).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(1809).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(2688).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3036).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(2174).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(4101).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3211).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3969).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(1375).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(290).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(1101).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(341).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3522).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3547).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(3567).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(4204).jpg',
 'Data/Data/PNEUMONIA/PNEUMONIA(190

In [4]:
import pandas as pd
import numpy as np

norm_split = list(pd.read_csv('normal_split_Caleb.csv', header = None)[0])
pnem_split = list(pd.read_csv('PNEUMONIA_split_Caleb.csv', header = None)[0])
covd_split = list(pd.read_csv('covid_split_Caleb.csv', header = None)[0])

In [5]:
print(norm_split[:5])
print(pnem_split[:5])
print(covd_split[:5])

['NORMAL(272).jpg', 'NORMAL(788).jpg', 'NORMAL(622).jpg', 'NORMAL(1138).jpg', 'NORMAL(1568).jpg']
['PNEUMONIA(1068).jpg', 'PNEUMONIA(3055).jpg', 'PNEUMONIA(1592).jpg', 'PNEUMONIA(1438).jpg', 'PNEUMONIA(3405).jpg']
['COVID19(309).jpg', 'COVID19(470).jpg', 'COVID19(535).jpg', 'COVID19(165).jpg', 'COVID19(92).jpg']


In [17]:
data_comb = []

for split in norm_split:
    data_comb.append([train_img[train_img.index('Data/Data/NORMAL/'+split)], \
                train_labels[train_img.index('Data/Data/NORMAL/'+split)]])

for split in pnem_split:
    data_comb.append([train_img[train_img.index('Data/Data/PNEUMONIA/'+split)], \
                train_labels[train_img.index('Data/Data/PNEUMONIA/'+split)]])
    
for split in covd_split:
    data_comb.append([train_img[train_img.index('Data/Data/COVID19/'+split)], \
                train_labels[train_img.index('Data/Data/COVID19/'+split)]])

In [28]:
print(train_img.index(train_img[0]))
print(len(data_comb))
print(data_comb[1])
print(data_comb[600])
print(data_comb[1500])

0
1609
['Data/Data/NORMAL/NORMAL(788).jpg', 0]
['Data/Data/PNEUMONIA/PNEUMONIA(354).jpg', 2]
['Data/Data/COVID19/COVID19(169).jpg', 1]


In [17]:
import sys
from pyspark.sql.functions import expr
from pyspark.sql import Window
import pyspark.sql.functions as psf
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pyspark.sql.functions as F


sc = pyspark.SparkContext.getOrCreate()

In [ ]:
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

image_df = spark.read.format('image').load(imgs)
print(image_df.printSchema())
image_df.select('image.data').show(2)
ImageSchema.imageFields
img2vec = F.udf(lambda x: DenseVector(ImageSchema.toNDArray(x).flatten()), VectorUDT())

df = image_df.withColumn('vecs', img2vec("image"))
df.show(1)

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

None


In [32]:

labels = spark.createDataFrame(data_comb, ['img', 'label'])
labels.show(5)

+--------------------+-----+
|                 img|label|
+--------------------+-----+
|Data/Data/NORMAL/...|    0|
|Data/Data/NORMAL/...|    0|
|Data/Data/NORMAL/...|    0|
|Data/Data/NORMAL/...|    0|
|Data/Data/NORMAL/...|    0|
+--------------------+-----+
only showing top 5 rows



In [10]:
def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    from pyspark.sql import Row
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("my_inception_udf", InceptionV3(weights="imagenet"), keras_load_img)

NameError: name 'registerKerasImageUDF' is not defined

In [ ]:
from keras.applications.resnet50 import ResNet50
model = ResNet50(weights=None)
model.save("path_to_my_model.h5")

In [ ]:
def load_image_and_process(uri):
    import PIL.Image
    from keras.applications.imagenet_utils import preprocess_input

    original_image = PIL.Image.open(uri).convert('RGB')
    resized_image = original_image.resize((224, 224), PIL.Image.ANTIALIAS)
    image_array = np.array(resized_image).astype(np.float32)
    image_tensor = preprocess_input(image_array[np.newaxis, :])
    return image_tensor